In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math

In [15]:
telecom_df = pd.read_csv('telecom_churn_data.csv')

In [16]:
telecom_df.info()
#Most are numeric, some of object type. Check these


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [17]:
#print(telecom_df.select_dtypes(include=['object']).head())
#These are all of type data, of format mm/dd/yyyy, conver to dates
#for all date columns, keep only the day of the month, since month is present in the
#column name, year is common for all.
date_columns = list(telecom_df.select_dtypes(include=['object']).columns)
for column in date_columns:
    telecom_df[column] = pd.to_datetime(telecom_df[column], format='%m/%d/%Y')
    telecom_df[column] = telecom_df[column].dt.day

In [18]:
#Those who have recharged with an amount more than or equal to X, 
#where X is the 70th percentile of the average recharge amount in the first two months (the good phase).
'''
print(telecom_df['total_rech_amt_6'].isnull().sum()) 0;
print(telecom_df['total_rech_amt_7'].isnull().sum()) 0;
print(telecom_df['total_rech_num_6'].isnull().sum()) 0;
print(telecom_df['total_rech_num_7'].isnull().sum()) 0;
'''
#since, all values for the columns used below are non-nulls, we need not do any imputation here
if 'av_rech_67' not in list(telecom_df.columns):
    telecom_df['av_rech_67'] =  (telecom_df['total_rech_amt_6']+telecom_df['total_rech_amt_7'])/(telecom_df['total_rech_num_6']+telecom_df['total_rech_num_7'])
    avg_70_quantile = telecom_df.av_rech_67.quantile(q=0.7)
    telecom_df = telecom_df[telecom_df.av_rech_67>= avg_70_quantile]
print ('Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: '+ str(len(telecom_df)))


Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: 29944


In [19]:
telecom_df.isnull().sum().describe()
# telecom_df.isnull().sum()>0).sum()
# 166 columns with missing values
# A lot of values are missing in many colums. Cannot remove such rows. Will have to find a way to impute

count      227.000000
mean      4205.832599
std       7207.678037
min          0.000000
25%          0.000000
50%        989.000000
75%       2083.000000
max      19826.000000
dtype: float64

In [20]:
#Find columns with only single non-null value
redundant_cols = [col for col in telecom_df.columns if telecom_df[col].nunique()==1]
telecom_df = telecom_df.drop(redundant_cols, axis=1)
print('Following columns were removed: \n',redundant_cols)


Following columns were removed: 
 ['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'last_date_of_month_9', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_og_t2c_mou_9', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8', 'std_ic_t2o_mou_9']


In [21]:
#Find rows with null dates
for month in [6,7,8,9]:
    last_rech_col = 'date_of_last_rech_'+str(month)
    num_recharges_col = 'total_rech_num_'+str(month)
    nullrechdate_df = telecom_df[telecom_df['date_of_last_rech_9'].isnull()]
    if (len(nullrechdate_df[~nullrechdate_df.total_rech_num_9==0])>0):
        print('Nulls found for date in month:', month)
    else:
        print('All nulls in month ',month, ' correspond to no recharges done.')
    
#Hence, nans in date_of_last_rech_ columns are all valid. Giving value > 32 so that we know it is invalid


#dates can be changed to categorical values

#NaN can be assumed to be Recharge not done 

# def process(x):
#     if type(x) is float:
#         return 0
#     else: 
#         return 1

# for col in dc:
#     dat[col] = dat[col].apply(process)

All nulls in month  6  correspond to no recharges done.
All nulls in month  7  correspond to no recharges done.
All nulls in month  8  correspond to no recharges done.
All nulls in month  9  correspond to no recharges done.


In [22]:
#Drop redundant columns
#For data, we have count and total_rech_data, 

for month in [6,7,8,9]:
    max_rech_data = 'max_rech_data_'+str(month)
    av_rech_amt_data='av_rech_amt_data_'+str(month)
    total_rech_data = 'total_rech_data_'+str(month)
    date_of_last_rech_data = 'date_of_last_rech_data_'+str(month)
    count_rech_3g = 'count_rech_3g_'+str(month)
    count_rech_2g = 'count_rech_2g_'+str(month)
    arpu_3g = 'arpu_3g_'+str(month)
    arpu_2g = 'arpu_2g_'+str(month)
    fb_user = 'fb_user_'+str(month)
    night_pck_user = 'night_pck_user_' +str(month)

    #Verify, that if toatl rech_data is null, then max, av, date, count_3g, count_2g are null too 
    nonnullrows = np.sum(np.sum(telecom_df[telecom_df[total_rech_data].isnull()]
                              [[max_rech_data,av_rech_amt_data,date_of_last_rech_data, count_rech_3g, count_rech_2g]]))
    print ("For month", month, "total non null/0 rows, when total recharge data is null are: ", nonnullrows)
    
    if nonnullrows == 0:
        telecom_df.loc[telecom_df[total_rech_data].isnull(),max_rech_data] = 0
        telecom_df.loc[telecom_df[total_rech_data].isnull(),av_rech_amt_data] = 0
        telecom_df.loc[telecom_df[total_rech_data].isnull(),date_of_last_rech_data] = 32
        telecom_df.loc[telecom_df[total_rech_data].isnull(),count_rech_3g] = 0
        telecom_df.loc[telecom_df[total_rech_data].isnull(),count_rech_2g] = 0    
        telecom_df.loc[telecom_df[total_rech_data].isnull(),arpu_3g] = 0    
        telecom_df.loc[telecom_df[total_rech_data].isnull(),arpu_2g] = 0    
        telecom_df.loc[telecom_df[total_rech_data].isnull(),fb_user] = 0
        telecom_df.loc[telecom_df[total_rech_data].isnull(),night_pck_user] = 0
        telecom_df.loc[telecom_df[total_rech_data].isnull(),total_rech_data] = 0    

For month 6 total non null/0 rows, when total recharge data is null are:  0.0
For month 7 total non null/0 rows, when total recharge data is null are:  0.0
For month 8 total non null/0 rows, when total recharge data is null are:  0.0
For month 9 total non null/0 rows, when total recharge data is null are:  0.0


In [23]:
(telecom_df.isnull().sum()>0).sum()

112

In [24]:
def makeZeroIfTotalIsZero(colm, total_colm, typeofcol):
    total_ = np.sum(telecom_df[telecom_df[colm].isnull()][total_colm])
    if total_ == 0:
        #('Total for ', typeofcol, ' is 0. Hence make all null values 0')
        telecom_df.loc[telecom_df[colm].isnull(),colm] = 0

for month in [6,7,8,9]:
    total_ic_mou = 'total_ic_mou_'+str(month)
    roam_ic_mou = 'roam_ic_mou_'+str(month)
    loc_ic_t2t_mou = 'loc_ic_t2t_mou_'+str(month)
    loc_ic_t2m_mou = 'loc_ic_t2m_mou_'+str(month)
    loc_ic_t2f_mou = 'loc_ic_t2f_mou_'+str(month)
    loc_ic_mou  = 'loc_ic_mou_'+str(month)
    std_ic_t2t_mou = 'std_ic_t2t_mou_'+str(month)
    std_ic_t2m_mou = 'std_ic_t2m_mou_'+str(month)
    std_ic_t2f_mou = 'std_ic_t2f_mou_'+str(month)
    std_ic_mou  =  'std_ic_mou_'+str(month)
    spl_ic_mou = 'spl_ic_mou_'+str(month)
    isd_ic_mou = 'isd_ic_mou_'+str(month)
    ic_others  = 'ic_others_'+str(month)

    total_og_mou = 'total_og_mou_' + str(month)
    roam_og_mou = 'roam_og_mou_' + str(month)
    loc_og_t2t_mou = 'loc_og_t2t_mou_' + str(month)
    loc_og_t2m_mou = 'loc_og_t2m_mou_' + str(month)
    loc_og_t2f_mou = 'loc_og_t2f_mou_' + str(month)
    loc_og_t2c_mou = 'loc_og_t2c_mou_' + str(month)
    loc_og_mou = 'loc_og_mou_' + str(month)
    std_og_t2t_mou = 'std_og_t2t_mou_' + str(month)
    std_og_t2m_mou = 'std_og_t2m_mou_' + str(month)
    std_og_t2f_mou = 'std_og_t2f_mou_' + str(month)
    std_og_mou = 'std_og_mou_' + str(month)
    isd_og_mou = 'isd_og_mou_' + str(month)
    spl_og_mou = 'spl_og_mou_' + str(month)
    og_others = 'og_others_' + str(month)
      
    onnet_mou = 'onnet_mou_'+str(month)
    offnet_mou = 'offnet_mou_'+str(month)

    date_of_last_rech = 'date_of_last_rech_' + str(month)
    total_rech_amt = 'total_rech_amt_'+ str(month)
    #Incoming columns
    for colm in [roam_ic_mou, loc_ic_t2t_mou, loc_ic_t2m_mou, 
                 loc_ic_t2f_mou, loc_ic_mou, std_ic_t2t_mou, 
                 std_ic_t2m_mou, std_ic_t2f_mou, std_ic_mou,
                 spl_ic_mou, isd_ic_mou, ic_others]:
        makeZeroIfTotalIsZero(colm, total_ic_mou, 'incoming')
    
    #Outgoing columns

    for colm in [roam_og_mou,loc_og_t2t_mou,loc_og_t2m_mou,
                 loc_og_t2f_mou,loc_og_t2c_mou,loc_og_mou,
                 std_og_t2t_mou,std_og_t2m_mou,std_og_t2f_mou,
                 std_og_mou,isd_og_mou,spl_og_mou,og_others]:
        makeZeroIfTotalIsZero(colm, total_ic_mou, 'outgoing')
    
    #Onnet columns
    #If outgoing + incoming mou for the nuxll entries is 0, then onnet, offnet mou are also 0
    outgoing_mou_when_onnet_null = np.sum(telecom_df[telecom_df[onnet_mou].isnull()][total_og_mou])
    incoming_mou_when_onnet_null = np.sum(telecom_df[telecom_df[onnet_mou].isnull()][total_ic_mou])
    onnet_null = outgoing_mou_when_onnet_null+incoming_mou_when_onnet_null
    #'Total onnet(calculated) for month: ', str(month), ' is: ', onnet, ' Actual null'..... #0
    if onnet_null == 0:
        telecom_df.loc[telecom_df[onnet_mou].isnull(),onnet_mou] = 0
    
    outgoing_mou_when_offnet_null = np.sum(telecom_df[telecom_df[offnet_mou].isnull()][total_og_mou])
    incoming_mou_when_offnet_null = np.sum(telecom_df[telecom_df[offnet_mou].isnull()][total_ic_mou])
    offnet_null = outgoing_mou_when_offnet_null+incoming_mou_when_offnet_null
    #'Total offnet(calculated) for month: ', str(month), ' is: ', offnet, ' Actual null'.... #0
    if offnet_null ==0:
        telecom_df.loc[telecom_df[offnet_mou].isnull(),offnet_mou] = 0
    
    total_ = np.sum(telecom_df[telecom_df[date_of_last_rech].isnull()][total_rech_amt])
    if total_ == 0:
        telecom_df.loc[telecom_df[date_of_last_rech].isnull(),date_of_last_rech] = 32

In [27]:
telecom_df.describe()

,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2f_mou_9,std_og_mou_6,std_og_mou_7,std_og_mou_8,std_og_mou_9,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,isd_og_mou_9,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,spl_og_mou_9,og_others_6,og_others_7,og_others_8,og_others_9,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_og_mou_9,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2t_mou_9,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2m_mou_9,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_t2f_mou_9,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,loc_ic_mou_9,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2t_mou_9,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2m_mou_9,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2f_mou_9,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,std_ic_mou_9,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_ic_mou_9,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,spl_ic_mou_9,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,isd_ic_mou_9,ic_others_6,ic_others_7,ic_others_8,ic_others_9,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_num_9,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,total_rech_amt_9,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9,total_rech_data_6,total_rech_data_7,total_rech_data_8,total_rech_data_9,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,count_rech_3g_9,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_3g_9,arpu_2g_6,arpu_2g_7,arpu_2g_8,arpu_2g_9,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,av_rech_67
count,2.994400e+04,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,29944.000000,2994

In [107]:
#Identify churners now
# total_ic_mou_9

# total_og_mou_9

# vol_2g_mb_9

# vol_3g_mb_9


/Users/aarora7/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2f_mou_9,std_og_mou_6,std_og_mou_7,std_og_mou_8,std_og_mou_9,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,isd_og_mou_9,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,spl_og_mou_9,og_others_6,og_others_7,og_others_8,og_others_9,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_og_mou_9,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2t_mou_9,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2m_mou_9,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_t2f_mou_9,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,loc_ic_mou_9,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2t_mou_9,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2m_mou_9,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2f_mou_9,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,std_ic_mou_9,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_ic_mou_9,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,spl_ic_mou_9,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,isd_ic_mou_9,ic_others_6,ic_others_7,ic_others_8,ic_others_9,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_num_9,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,total_rech_amt_9,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9,total_rech_data_6,total_rech_data_7,total_rech_data_8,total_rech_data_9,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,count_rech_3g_9,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_3g_9,arpu_2g_6,arpu_2g_7,arpu_2g_8,arpu_2g_9,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,av_rech_67
0,7000842753,197.385,214.816,213.803,21.100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.16,0.0,0.00,0.00,4.13,0.0,0.00,0.00,1.15,0.0,0.00,0.00,5.44,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,5.44,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,4,3,2,6,362,252,252,0,252,252,252,0,21.0,16.0,8.0,28.0,252,252,252,0,21.0,16.0,8.0,32.0,1.0,1.0,1.0,0.0,252.0,252.0,252.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,252.0,252.0,252.0,0.0,30.13,1.32,5.75,0.00,83.57,150.76,109.61,0.00,212.17,212.17,212.17,0.00,212.17,212.17,212.17,0.00,0.0,0.0,0.0,0.0,0,0